# Squelette du TP3 -- Approches contenu et techniques de graphes
Le lien vers l'énoncé est [ici](https://cours.polymtl.ca/MDesmarais/log6308/Tp/20231/tp3.html)

In [ ]:
!pip install networkx==3.0 scipy==1.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
""""
Squelette du TP3 du cours LOG6308 donné par le Professeur Desmarais.
Écrit par Jean-Charles Layoun le 19 février 2023.
"""

import numpy as np
import pandas as pd 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import networkx as nx

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# --------------------------- Définition des Fonctions ---------------------------
def MRR_on_ranks(query_ranks):
    "Takes an array of ranks of the query set and returns the MRR of that query set."
    return np.mean(1/np.array(query_ranks))

In [ ]:
## Defining Figure Size:
sns.set(rc={"figure.figsize":(12, 6)})

#### Loading the Data

In [ ]:
## Loading the Adjacency Matrix, node IDs, test dataset and Creating graph
M             = scipy.io.mmread('Data/tp-matrix.dgt').tocsr()
G             = nx.from_numpy_array(M, create_using=?)# Fill with the right 
test_articles = pd.read_csv('Data/12-articles.csv')
node_ids_csv  = pd.read_csv('Data/tp-matrix-names.csv', sep="\s+")
node_ids_csv.rename(columns={"x": "id"}, inplace=True)

#### Showing the data

In [ ]:
test_articles.head()

,id,titre
1,53e9aa4fb7602d97033bee00,Joint Energy Management and Resource Allocatio...
2,53e9b098b7602d9703b03c4e,Qualitative organization of collections of sha...
3,53e9b708b7602d97042a150f,Using the fuzzy multi-criteria decision making...
4,53e9ba6ab7602d970468b7d5,Multi-Scale Adaptive Sampling with Mobile Agen...
5,53e9ad6cb7602d9703755d9b,Randomized locality sensitive vocabularies for...


In [ ]:
node_ids_csv.head()

,id
1,53e99808b7602d970201b1fe
2,53e9b37bb7602d9703e5eb33
3,53e9b917b7602d9704500623
4,56d90f35dabfae2eee308269
5,53e9af87b7602d97039ce689


In [ ]:
test_articles.iloc[0]

id                                53e9aa4fb7602d97033bee00
titre    Joint Energy Management and Resource Allocatio...
Name: 1, dtype: object

On observe un décalage entre les indices des sommets dans le CSV et dans le Graphe **G**

In [ ]:
list(G.nodes())[0]

0

In [ ]:
node_ids_csv.index[0]

1

In [ ]:
## Getting the indices of test articles in the Graph
test_indices = ?
test_indices

Int64Index([ 1593, 11635, 12214, 12592, 15088, 18538, 18644, 35303, 35352,
            36564, 50495, 50496],
           dtype='int64')

## Q1 : Page Rank Global

In [ ]:
# Computing the Global Pagerank via NetworkX
pr              = nx.pagerank(?, alpha=0.85)#alpha is the damping factor
sorted_articles = ?

In [ ]:
# Exemple qui montre qu'il faut rajouter +1
np.where(sorted_articles == 32)[0][0]+1

1

In [ ]:
# Computing MRR for each article in test set:
MRR_list = []
for i, test_index in enumerate(test_indices):

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.002027
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.066947
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.002276
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.002594
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.000455
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.000145
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.002117
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.001798
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

## Q2 : Page Rank Thématique 
Le lien à l'explication du Page Rank Thématique : [lien](https://www.youtube.com/watch?v=URaS1u-Murc&ab_channel=ArtificialIntelligence-AllinOne).

Soit $\text{Pr}_{i,j}$ la probabilité de transition entre le sommet $i$ et le sommet $j$, $A$ la matrice d'adjacence, et $S$ l'ensemble thématique. On peut exprimer $\text{Pr}_{i,j}$ ainsi :<br>

\begin{equation}
  {Pr}_{i,j} =
    \begin{cases}
      \beta \cfrac{A_{i,j}}{\underset{j}{\sum}A_{i,j}} + \cfrac{(1 - \beta)}{|S|} & \text{si $j \in S$}\\
      \beta \cfrac{A_{i,j}}{\underset{j}{\sum}A_{i,j}} & \text{Sinon.}
    \end{cases}       
\end{equation}


Remarques :
- La matrice de probabilité de transition $\text{Pr}$ est stochastique. En effet, la somme des lignes de $\text{Pr}$ est égale à $1$. 
- La valeur $\cfrac{(1 - \beta)}{|S|}$ qu'on rajoute aux probabilités des sommets de l'ensemble thématique symbolise qu'à tout moment on peut se téléporter dans cet ensemble avec une probabilité de $\cfrac{(1 - \beta)}{|S|}$.

In [ ]:
# --------------------------- Définition des Fonctions pour l'Approche Thématique ---------------------------
def ComputeThematicSubspace(M_adj, node_idx):
    ordre_1    = M_adj[node_idx]                  #Articles liés par la relation 1
    ordre_n1   = M_adj.T[node_idx]                #Articles liés par la relation -1
    ordre_11   = ordre_1.dot(M_adj)               #Articles liés par la relation (1, 1)
    ordre_1n1  = ?             #Articles liés par la relation (1, -1)
    ordre_n11  = ?              #Articles liés par la relation (-1, 1)
    ordre_1n11 = ?  #Articles liés par la relation (1, -1, 1)
    ordre_n111 = ?   #Articles liés par la relation (-1, 1, 1)
    
    # Creation de l'ensemble thématique
    thematic   = ordre_1+ordre_n1+ordre_11+ordre_1n1+ordre_1n11+ordre_1n11+ordre_n111

    # Conversion en vecteur binaire d'adjacence
    thematic[thematic>1] = 1

    return thematic

def ThematicPageRank(M_adj, node_idx, Beta=0.80, max_iter=1000, epsilon=1e-5):
    #initialize page rank
    r_prev   = np.array([[1]]*M_adj.shape[0])
    teleport = ComputeThematicSubspace(M_adj, node_idx)
    for i in range(max_iter):
        #to avoid division by 0, the denominator  increases by 1
        r_cur=?
        if np.mean(np.abs(r_cur-r_prev))<epsilon:
            #to know the number of iteration to achive convergence we print i
            #print(f'number of iteration was: {i}')
            break
        else:
            r_prev=r_cur

    return r_cur


In [ ]:
# Computing MRR for each article in test set with nx.pagerank:
MRR_list_q2 = []
for i, test_index in enumerate(test_indices):

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list_q2)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.118467
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.085281
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.074370
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.072830
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.050930
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.035758
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.021833
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.089124
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

In [ ]:
# Computing MRR for each article in test set with custom PR function:
MRR_list_q2_C = []
for i, test_index in enumerate(test_indices):

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list_q2_C)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.101444
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.076640
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.073505
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.067089
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.021535
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.023665
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.019624
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.081205
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

## Q3 : Aprroche Similarité Cosinus

In [ ]:
cos_sim_test_adjmat    = cosine_similarity(?, ?)

In [ ]:
## On remarque que la similarité cosinus entre deux vecteurs identiques est ~1
cos_sim_test_adjmat[:, test_indices]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
# Computing MRR for each article in test set:
print("\t\t\tRéstultats de l'approche Similarité Cosinus : \n")

MRR_list_q3 = []
for i, test_index in enumerate(test_indices):

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list_q3)))

			Réstultats de l'approche Similarité Cosinus : 

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.058316
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.001257
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.001830
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.010049
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.014697
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.019227
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.013581
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.008132
Le MRR de l'article `Dist